# Use databases and dataviz tools to empower analysis

In this notebook we will get data from MinIO bucket, insert it into a database table and visualize outputs on an interface

### 1.1.0 Get data from MinIO

With the last notebook, create a minio client, get your parquet file and read it with pandas

In [ ]:
#%pip install pyarrow

In [ ]:
# import dependancies
from minio import Minio
import urllib3
import pandas as pd
import pyarrow.parquet as pq
from io import BytesIO, StringIO


In [ ]:
## Create a client with the access key and the secret key given
import os
client = Minio(
    ...
)

In [ ]:
### path to the object into minIO
path_minio=...

### Reuse your bucket name
bucket=''#name-surname

In [ ]:
# Get data from minio using get_object, decode it using BytesIO and read the parquet result with pandas
try:
    response = client....
    # Read data from response.
    parquet_object=...
    data = pd....
finally:
    response.close()
    response.release_conn()

In [ ]:
# Take a look at the data
data....

### 1.1.1 Clickhouse create db and tables

Here we want to store our numerical features for futur analysis / model training / preprocessing.

In [ ]:
#install dependancies : pandahouse, pandas client to talk with clickhouse 
%pip install pandahouse

In [ ]:
# import depandancies
import pandahouse as ph

#### Create clickhouse connection

In [ ]:
## The connection dict need a default database
connection = dict(database='default',
                  host='http://clickhouse-install.clickhouse.svc.cluster.local:8123',
                  user='admin',
                  password='B1gdata-demo')

In [ ]:
# execute a show databases command to verify the connection is working
ph.read_clickhouse('...',connection=connection)

In [ ]:
### helper function for handle this python client
def write_clickhouse(query,connection):
    print(query)
    try:
        ph.read_clickhouse(query,connection=connection)
    except KeyError:
        print("Nothing to return")

**Create a db named firstname-lastname, as in your credentials**

In [ ]:
# firstname_lastname, as in your credentials but with "_" instead of "-" because clickhouse does not allow "-" in db name
dbname = ''#name_surname

In [ ]:
### create a personal database if it does not exists
write_clickhouse(f" ... ",connection)

In [ ]:
### override connection dict with personal database
connection['database'] = f"{dbname}"

In [ ]:
# verify the connection content is related to your user
connection

#### Create clickhouse table taxi_trips

In [ ]:
dbtable='chicago_taxi'

In [ ]:
### select features
features = data[[
    "tips",
    "trip_start_timestamp",
    "trip_seconds",
    "trip_miles",
    "pickup_community_area" ,
    "dropoff_community_area" ,
    "fare",
    "tolls",
    "extras",
    "trip_total"
]]

In [ ]:
### create table for inserting taxi trip dataset 
## Clickhouse table definition
# using the df informations, and clickhouse documentation write  the create table statement
taxitable = f"""
CREATE TABLE IF NOT EXISTS {dbname}.{dbtable}
(
    ...
) 
ENGINE = MergeTree
PARTITION BY toYYYYMM(trip_start_timestamp)
ORDER BY trip_start_timestamp;
"""

In [ ]:
write_clickhouse(...,connection)

#### Insert the data into taxi_trips table

In [ ]:
## We have to be compliant with the clickhouse date type. 
## we need to force '%Y-%m-%d %H:%M:%S'
## force the date format with the defined schema, using pandas
features["trip_start_timestamp"] = pd....

In [ ]:
### insert using the to_clickhouse function
ph.to_clickhouse(features, dbtable, index=False, chunksize=100000, connection=connection)

### 1.1.2 Postgresql Create db and table

Here we want to store a referential of pickup community area and related long / lat. To feed future map analysis.

In [ ]:
%pip install psycopg2-binary

In [ ]:
# import depandancies : psycopg2-binary
import psycopg2


In [ ]:
# From the data we create a de-deduplicated, non null value referential
scope = data[[
    "pickup_community_area",
    "pickup_centroid_latitude",
    "pickup_centroid_longitude"
    ]].drop_duplicates().dropna()

In [ ]:
# verify that the length of the referential is coherent
len(...)

In [ ]:
# head some lines of the scoped data
scope....

#### Create postgres connection

In [ ]:
# postgres is the default database, autocommit enable folder level actions
conn = psycopg2.connect(
   database="postgres", user='postgres', password='B1gdata-demo', host='mypostgres.kubegres.svc.cluster.local', port= '5432'
)
conn.autocommit = True
cursor = conn.cursor()

#### Create postgres personal DB

In [ ]:
## Define the postgres database name
# firstname_lastname, as in your credentials but with "_" instead of "-" because postgres does not allow "-" in db name
dbname = ''#name_surname

In [ ]:
# write the query to create a new database named with the dbname var
sqlCreateDb = f""" create database {dbname}"""
# execute the query using the cursor
cursor.execute(sqlCreateDb)

In [ ]:
## Now overwrite the conn with your personnal DB
conn = psycopg2.connect(
   database=dbname, user='postgres', password='B1gdata-demo', host='mypostgres.kubegres.svc.cluster.local', port= '5432'
)
conn.autocommit = True
cursor = conn.cursor()

#### Create table in postgres

In [ ]:
### set table name
table_name='chicago_areas'

In [ ]:
### Find the right way to define the table using postgresql documentation
# focus on schema and types
# using the df informations, and postgres documentation write the create table statement

areas_table=f"""
CREATE TABLE IF NOT EXISTS {table_name} (
   ...)
"""

In [ ]:
## execute the table creation query
cursor.execute(...)

#### Insert data into table

In [ ]:
def insert_df_to_table(df,table,conn,cursor):
    """
    insert data to postgres table from pandas dataframe
    """
    # prepare object to stream data
    output = StringIO()
    # put data into StringIO object as a csv 
    df.to_csv(output, sep='\t', header=False, index=False)
    # replace output cursor position  position 0
    output.seek(0)
    # copy content from stream object to table
    cursor.copy_from(output, table, null="") # null values become ''
    conn.commit()

In [ ]:

insert_df_to_table(...,...,...,...)

#### Verify if content is loaded properly

In [ ]:
# define a select statement to get 5 top records of your areas table
selectexp = f" ..."

In [ ]:
# query the base and return a Pandas dataframe using read_sql_query function from pandas
frame = pd.read_sql_query(...,conn)

In [ ]:
# Check the 5 rows you select
frame.head()

### 1.1.3 Use kafka brokers and topics to send your data event by event

In [ ]:
### persist Data locally to source the stream
data.to_csv("./chicagodata/to_be_sent_into_kafka.csv")

## Producer

#### Configure a producer that will send events to kafka

In [ ]:
topicName = 'aiengineer.tp.1' # this is the topic where we will produce events
kafkabrokers="https://streaming-bridge.course.aiengineer.codex-platform.com" #the link to the messaging service
headers={'content-type': 'application/vnd.kafka.binary.v2+json'} #metadata to define the message format/encoding
keyName=''#name-surname   #the key you will use to get the data when consuming

#### Send each line from the dataset

### Before executing the next cells, open and execute the last notebook : [2_receive_stream_data.ipynb](./2_receive_stream_data.ipynb)

In [ ]:
# install dependancies : a python client that allow writing events to our kafka bridge
%pip install kafka_bridge_client

In [ ]:
# import dependancies
from kafka_bridge_client import KafkaBridgeProducer,Message

In [ ]:
# create an instance of the producer, that will connect and write to the messaging system
producer = KafkaBridgeProducer(kafkabrokers,timeout=5)

In [ ]:
# stram our chicago dataset line per line in bytes format to the messaging system
with open("./chicagodata/to_be_sent_into_kafka.csv") as f:
    for i, line in enumerate(f):
        # only send the 10 first lines, exept the header in position 0
        if i > 0 and i < 10:
            producer.send(
                topic=topicName,
                # Create a Message object containing our key and our line byte encoded (src encoding is 'utf-8')
                record = Message(key=keyName,value=bytes(line,'utf-8')),
                binary=True
            )

### 1.1.4 Visualize on superset

Go to [https://dataviz.course.aiengineer.codex-platform.com/](https://dataviz.course.aiengineer.codex-platform.com/) (or look for superset in product portal) and log with your account

on Data > Databases you should see a database named `clickhouse`. This will be our source

![source](./images/source.png)

With this source we will create a superset dataset. It maps a table and allow exploration/ chart creation using it

![dataset](./images/dataset.png)

![table](./images/table.png)

One you choose the dataset, click on it and start create some charts

![tips](./images/tips.png)

**In this example**

- Chart type is bar chart
- No time range because dataset has old dates values
- metric is average tips (y)
- serie is pickup location (x)

You can name, save and assign chart to a dashboard.


**Go further : Create a dashboard with multiple vizualisation answering to feature analysis, try to represent the dataset on a map**